In [5]:
import pandas as pd
import numpy as np
import os
import json
import pickle
from collections import defaultdict
from datetime import datetime
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored
openai.api_key = os.environ.get('OPENAI_API_KEY')

In [6]:
from openai import OpenAI
load_dotenv()
client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [7]:
user_test_path = '../fashion_data/processed_data/user_test.json'
user_valid_path = '../fashion_data/processed_data/user_valid.json'
user_train_path = '../fashion_data/processed_data/user_train.json'

In [8]:
with open(user_test_path, 'rb') as f:
    user_test_dict = json.load(f)
print(len(user_test_dict))

1273


In [9]:
with open(user_valid_path, 'rb') as f:
    user_valid_dict = json.load(f)
print(len(user_valid_dict))

1273


In [10]:
with open(user_train_path, 'rb') as f:
    user_train_dict = json.load(f)
print(len(user_train_dict))

1273


In [12]:
user_id = '1'
print(user_train_dict[user_id])
print(user_valid_dict[user_id])
print(user_test_dict[user_id])

[11, 12, 13, 14, 15]
[12, 13, 14, 15, 16]
[13, 14, 15, 16, 17]


In [14]:
len(user_train_dict[user_id])

5

In [15]:
item_information_path = '../fashion_data/processed_data/processed_item_information.json'
with open(item_information_path, 'r') as f:
    item_information_dict = json.load(f)
print(len(item_information_dict))

6089


In [18]:
print(item_information_dict['0'])

{'title': 'Allegra K Lady Long Sleeve Letter Pattern Pullover Knit Shirt Black S', 'description': '\n\nThe product is a cute and comfy t-shirt, fitting well with leggings, and available in various sizes. It has a good quality fabric and length, but is very casual and simple.\n\nNote: The product description is written in 25 words as requested, and it highlights both'}


In [16]:
ratings_path = '../fashion_data/processed_data/ratings.csv'
ratings_df = pd.read_csv(ratings_path)
ratings_df.head()

,item,user,rating,timestamp
0,0,0,4.0,1452816000
1,1,0,4.0,1453766400
2,2,0,3.0,1453766400
3,3,0,4.0,1453766400
4,4,0,4.0,1453766400


In [14]:
train_data = []
instruction = "Given the user preference and unpreference, identify whether the user will like the target movie by answering \"Yes.\" or \"No.\"."
# print(instruction)
cnt = 0
for user, user_profile in user_profile_dict.items():
    user_dict = dict()
    user_dict["instruction"] = instruction
    cnt += 1
    liked_movies = []
    disliked_movies = []
    user_rating_dict = ratings_df[ratings_df['user_id'] == user]
    # print(user_rating_dict)
    for movie in user_train_dict[user][0][-21:-1]:
        # print(movie)
        if user_rating_dict[user_rating_dict['movie_id'] == movie]['rating'].values[0] >= 3:
            liked_movies.append(movie)
        else:
            disliked_movies.append(movie)
    input = ""
    if len(liked_movies) > 0:
        input += "\nUser Preference:"
        for movie in liked_movies:
            movie_details = movie_summary_df[movie_summary_df['movie_id'] == movie]
            input += f" \"{movie_details['name'].values[0]}({str(movie_details['year'].values[0])})\","
    if len(disliked_movies) > 0:
        input += "\nUser Unpreference:"
        for movie in disliked_movies:
            movie_details = movie_summary_df[movie_summary_df['movie_id'] == movie]
            input += f"\"{movie_details['name'].values[0]}({str(movie_details['year'].values[0])})\","
    movie_details = movie_summary_df[movie_summary_df['movie_id'] == user_train_dict[user][0][-1]]
    input += f"\n Whether the user will like the target movie \"{movie_details['name'].values[0]}({str(movie_details['year'].values[0])})\?"
    # print(input)
    user_dict["input"] = input
    if user_rating_dict[user_rating_dict['movie_id'] == user_train_dict[user][0][-1]]['rating'].values[0] >= 3:
        user_dict["output"] = "Yes."
    else:
        user_dict["output"] = "No."
    train_data.append(user_dict)
    # if cnt == 5:
    #     print(train_data)
    #     break
print(len(train_data))

6040


In [ ]:
train_data = []
instruction = "Given the user preference and unpreference, identify whether the user will like the target item by answering \"Yes.\" or \"No.\"."
# print(instruction)
cnt = 0
for user, item_list in user_test_dict.items():  
    # print(user, item_list)
    user_rating_dict = ratings_df[ratings_df['user'] == int(user)]
    

In [15]:
valid_data = []
instruction = "Given the user preference and unpreference, identify whether the user will like the target movie by answering \"Yes.\" or \"No.\"."
# print(instruction)
cnt = 0
for user, user_profile in user_profile_dict.items():
    user_dict = dict()
    user_dict["instruction"] = instruction
    cnt += 1
    liked_movies = []
    disliked_movies = []
    user_rating_dict = ratings_df[ratings_df['user_id'] == user]
    # print(user_rating_dict)
    for movie in user_train_dict[user][0][-20:]:
        # print(movie)
        if user_rating_dict[user_rating_dict['movie_id'] == movie]['rating'].values[0] >= 3:
            liked_movies.append(movie)
        else:
            disliked_movies.append(movie)
    input = ""
    if len(liked_movies) > 0:
        input += "\nUser Preference:"
        for movie in liked_movies:
            movie_details = movie_summary_df[movie_summary_df['movie_id'] == movie]
            input += f" \"{movie_details['name'].values[0]}({str(movie_details['year'].values[0])})\","
    if len(disliked_movies) > 0:
        input += "\nUser Unpreference:"
        for movie in disliked_movies:
            movie_details = movie_summary_df[movie_summary_df['movie_id'] == movie]
            input += f"\"{movie_details['name'].values[0]}({str(movie_details['year'].values[0])})\","
    
    movie_details = movie_summary_df[movie_summary_df['movie_id'] == user_valid_dict[user][0]]
    input += f"\n Whether the user will like the target movie \"{movie_details['name'].values[0]}({str(movie_details['year'].values[0])})\?"
    # print(input)
    user_dict["input"] = input
    if user_rating_dict[user_rating_dict['movie_id'] == user_valid_dict[user][0]]['rating'].values[0] >= 3:
        user_dict["output"] = "Yes."
    else:
        user_dict["output"] = "No."
    valid_data.append(user_dict)
    # if cnt == 5:
    #     print(valid_data)
    #     break
print(len(valid_data))

6040


In [16]:
print(train_data[5])
print(valid_data[5])

{'instruction': 'Given the user preference and unpreference, identify whether the user will like the target movie by answering "Yes." or "No.".', 'input': '\nUser Preference: "People vs. Larry Flynt, The (1996)", "Cider House Rules, The (1999)", "JFK (1991)", "Dances with Wolves (1990)", "Private Parts (1997)", "Amistad (1997)", "Outbreak (1995)", "Life and Times of Hank Greenberg, The (1998)", "Remember the Titans (2000)", "Amadeus (1984)", "Right Stuff, The (1983)", "Roger & Me (1989)", "When Harry Met Sally... (1989)", "Do the Right Thing (1989)", "Local Hero (1983)", "Moonstruck (1987)", "Fish Called Wanda, A (1988)", "She\'s Gotta Have It (1986)",\nUser Unpreference:"Horse Whisperer, The (1998)","Any Given Sunday (1999)",\n Whether the user will like the target movie "Thin Blue Line, The (1988)\\?', 'output': 'Yes.'}
{'instruction': 'Given the user preference and unpreference, identify whether the user will like the target movie by answering "Yes." or "No.".', 'input': '\nUser Pre

In [17]:
test_data = []
instruction = "Given the user preference and unpreference, identify whether the user will like the target movie by answering \"Yes.\" or \"No.\"."
# print(instruction)
cnt = 0
for user, user_profile in user_profile_dict.items():
    user_dict = dict()
    user_dict["instruction"] = instruction
    cnt += 1
    liked_movies = []
    disliked_movies = []
    user_rating_dict = ratings_df[ratings_df['user_id'] == user]
    # print(user_rating_dict)
    movie_history = user_train_dict[user][0][-19:]
    movie_history.append(user_valid_dict[user][0])
    # print("Movie History Length: ",len(movie_history))
    for movie in movie_history:
        # print(movie)
        if user_rating_dict[user_rating_dict['movie_id'] == movie]['rating'].values[0] >= 3:
            liked_movies.append(movie)
        else:
            disliked_movies.append(movie)
    input = ""
    if len(liked_movies) > 0:
        input += "\nUser Preference:"
        for movie in liked_movies:
            movie_details = movie_summary_df[movie_summary_df['movie_id'] == movie]
            input += f" \"{movie_details['name'].values[0]}({str(movie_details['year'].values[0])})\","
    if len(disliked_movies) > 0:
        input += "\nUser Unpreference:"
        for movie in disliked_movies:
            movie_details = movie_summary_df[movie_summary_df['movie_id'] == movie]
            input += f"\"{movie_details['name'].values[0]}({str(movie_details['year'].values[0])})\","
    
    movie_details = movie_summary_df[movie_summary_df['movie_id'] == user_test_dict[user][0]]
    input += f"\n Whether the user will like the target movie \"{movie_details['name'].values[0]}({str(movie_details['year'].values[0])})\?"
    # print(input)
    user_dict["input"] = input
    if user_rating_dict[user_rating_dict['movie_id'] == user_test_dict[user][0]]['rating'].values[0] >= 3:
        user_dict["output"] = "Yes."
    else:
        user_dict["output"] = "No."
    test_data.append(user_dict)
    # if cnt == 1:
    #     print(test_data)
    #     break
print(len(test_data))

6040


In [18]:
print(len(train_data))
print(len(valid_data))
print(len(test_data))

6040
6040
6040


In [19]:
print(train_data[0])
print(valid_data[0])
print(test_data[0])

{'instruction': 'Given the user preference and unpreference, identify whether the user will like the target movie by answering "Yes." or "No.".', 'input': '\nUser Preference: "James and the Giant Peach (1996)", "Ferris Bueller\'s Day Off (1986)", "Secret Garden, The (1993)", "Toy Story 2 (1999)", "Airplane! (1980)", "Dumbo (1941)", "Pleasantville (1998)", "Princess Bride, The (1987)", "Snow White and the Seven Dwarfs (1937)", "Miracle on 34th Street (1947)", "Ponette (1996)", "Schindler\'s List (1993)", "Toy Story (1995)", "Aladdin (1992)", "Tarzan (1999)", "Close Shave, A (1995)", "Beauty and the Beast (1991)", "Antz (1998)", "Hunchback of Notre Dame, The (1996)", "Bug\'s Life, A (1998)",\n Whether the user will like the target movie "Mulan (1998)\\?', 'output': 'Yes.'}
{'instruction': 'Given the user preference and unpreference, identify whether the user will like the target movie by answering "Yes." or "No.".', 'input': '\nUser Preference: "Ferris Bueller\'s Day Off (1986)", "Secret

In [20]:
train_valid_data = train_data.copy()
train_valid_data.extend(valid_data)
print(len(train_valid_data), train_valid_data[:3])
print(len(train_data), len(valid_data), len(test_data))

12080 [{'instruction': 'Given the user preference and unpreference, identify whether the user will like the target movie by answering "Yes." or "No.".', 'input': '\nUser Preference: "James and the Giant Peach (1996)", "Ferris Bueller\'s Day Off (1986)", "Secret Garden, The (1993)", "Toy Story 2 (1999)", "Airplane! (1980)", "Dumbo (1941)", "Pleasantville (1998)", "Princess Bride, The (1987)", "Snow White and the Seven Dwarfs (1937)", "Miracle on 34th Street (1947)", "Ponette (1996)", "Schindler\'s List (1993)", "Toy Story (1995)", "Aladdin (1992)", "Tarzan (1999)", "Close Shave, A (1995)", "Beauty and the Beast (1991)", "Antz (1998)", "Hunchback of Notre Dame, The (1996)", "Bug\'s Life, A (1998)",\n Whether the user will like the target movie "Mulan (1998)\\?', 'output': 'Yes.'}, {'instruction': 'Given the user preference and unpreference, identify whether the user will like the target movie by answering "Yes." or "No.".', 'input': '\nUser Preference: "Terminator, The (1984)", "Winnie t

### Train Valid split 70:30

In [22]:
import random
random_seed = 42
random.seed(random_seed)
random.shuffle(train_valid_data)

train_valid_split = 0.7
split_index = int(train_valid_split*len(train_valid_data))
print(split_index)

8456


In [23]:
new_train_data = train_valid_data[:split_index]
new_valid_data = train_valid_data[split_index:]

print("Train Data: ", len(new_train_data), new_train_data[:3])
print("Valid Data: ", len(new_valid_data), new_valid_data[:3])

Train Data:  8456 [{'instruction': 'Given the user preference and unpreference, identify whether the user will like the target movie by answering "Yes." or "No.".', 'input': '\nUser Preference: "Papillon (1973)", "Buddy Holly Story, The (1978)", "Klute (1971)", "Kramer Vs. Kramer (1979)", "Rocky (1976)", "Way We Were, The (1973)", "Midnight Express (1978)", "Candidate, The (1972)", "Last Detail, The (1973)", "Carnal Knowledge (1971)", "Tin Drum, The (Blechtrommel, Die) (1979)", "...And Justice for All (1979)", "Coming Home (1978)", "Death Wish (1974)", "Towering Inferno, The (1974)", "Rocky II (1979)", "North Dallas Forty (1979)", "Airport (1970)", "Airport 1975 (1974)", "Airport \'77 (1977)",\n Whether the user will like the target movie "Concorde: Airport \'79, The (1979)\\?', 'output': 'Yes.'}, {'instruction': 'Given the user preference and unpreference, identify whether the user will like the target movie by answering "Yes." or "No.".', 'input': '\nUser Preference: "Donnie Brasco (

### Storing as json

In [24]:
import json

with open('./data/train.json', 'w') as train_file:
    json.dump(new_train_data, train_file)

with open('./data/valid.json', 'w') as valid_file:
    json.dump(new_valid_data, valid_file)

with open('./data/test.json', 'w') as test_file:
    json.dump(test_data, test_file)

In [28]:
content = test_data[4]["instruction"] + test_data[4]["input"]
print(content)
print('*'*100)
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "user",
      "content": content
    }
  ],
  temperature=0.1,
  max_tokens=64,
  top_p=0.9
)

response = response.choices[0].message.content
print(response)
len(response.split(' '))

Given the user preference and unpreference, identify whether the user will like the target movie by answering "Yes." or "No.".
User Preference: "Picture Perfect (1997)", "Tango Lesson, The (1997)", "Wilde (1997)", "Wings of the Dove, The (1997)", "Crash (1996)", "Kama Sutra: A Tale of Love (1996)", "Ponette (1996)", "Up Close and Personal (1996)", "Fantasia (1940)", "Joe Gould's Secret (2000)", "Aimée & Jaguar (1999)", "King of Masks, The (Bian Lian) (1996)", "Yards, The (1999)", "Goya in Bordeaux (Goya en Bodeos) (1999)", "Wonder Boys (2000)",
User Unpreference:"Tomorrow Never Dies (1997)","Replacements, The (2000)","Autumn in New York (2000)","Boys and Girls (2000)","Golden Bowl, The (2000)",
 Whether the user will like the target movie "Courage Under Fire (1996)\?
****************************************************************************************************
Yes.


1